# Starting and accessing the store

In [ ]:
import subprocess
from improv.store import Limbo

In [ ]:
# set up the store
p = subprocess.Popen(["/home/ameliacang7/anaconda3/envs/improv/bin/plasma_store", 
                      "-s", "/tmp/store", "-m", str(10000000)],
                     \stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [ ]:
# set up the store interface with Limbo class
lmb = Limbo(store_loc = "/tmp/store")

In [ ]:
# checking put and get functions
id = lmb.put(37, "this is 37") #putting this message into the store, located at id
val = lmb.getID(id) #getting the id from the store and retrieving the value
val

In [ ]:
# make sure to kill the process when you are done
p.kill()

# Example server and client messaging with ZMQ

In [ ]:
import zmq
import sys

In [ ]:
# setting up the jupyter notebook as the client

port = "5556"
context = zmq.Context()
print("Connecting to server...")
socket = context.socket(zmq.REQ)
socket.connect ("tcp://localhost:%s" % port)

Before the messages between the server and the client start, the server file (python example_server.py) must be ran. In another terminal besides for the terminal used for this Jupyter Notebook, run `python example_server.py`.

Afterwards, run the next cell to see the messages being sent between the client and the server.


In [ ]:
# messaging between client and server over 10 requests

# sending 10 requests to the server
for request in range (1,10):
    print("Sending request ", request,"...")
    socket.send_string("Hello")
    #  get the reply
    message = socket.recv()
    print("Received reply ", request, "[", message, "]")

# Attemping to load image data live with ZMQ

In [ ]:
import zmq
import sys
import numpy as np
import matplotlib.pylab as plt
import math
import time
from IPython.display import clear_output
import pyarrow.plasma as plasma

import subprocess
from improv.store import Limbo

In [ ]:
# setting up the socket and connecting to the store

lmb = Limbo(store_loc = "/tmp/store")
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect("tcp://127.0.0.1:5555")
socket.setsockopt_string(zmq.SUBSCRIBE, '')

Before running the next cell, the server needs to be started. In another terminal aside from the terminal for this Jupyter Notebook, run `python acq_zmq.py`.

Now, the following cells can be ran

In [ ]:
# receiving messages from server (acq_zmq.py)
n=1
print("Waiting to receive message from zmq ...")
while True:
    clear_output(wait=True)
    message = socket.recv()
    print(n, message)
    n = n+1

In [ ]:
# accessing id and data from store
import pyarrow.plasma as plasma
dataobj = plasma.ObjectID(message)
data = lmb.getID(dataobj)
data

In [ ]:
plt.imshow(data, cmap='inferno')

With this code, we are receiving live messages from the server, acq_zmq.py. However, the image isn't updating properly. The Jupyter Notebook `data_loading` has the working code.